<a href="https://colab.research.google.com/github/Vinayyarakalapudi/miniproject/blob/main/price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello")

hello


In [2]:
!pip install scikit-learn pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
df = pd.read_csv("crop_price_dataset.csv")

df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

df.head()

In [ ]:
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

df = df[(df['Price'] >= Q1 - 1.5*IQR) &
        (df['Price'] <= Q3 + 1.5*IQR)]

In [ ]:
df['Month'] = df['Date'].dt.month

df['Lag1'] = df['Price'].shift(1)
df['Lag2'] = df['Price'].shift(2)
df['Lag3'] = df['Price'].shift(3)

df['Rolling_Mean_3'] = df['Price'].rolling(3).mean()
df['Rolling_Mean_6'] = df['Price'].rolling(6).mean()
df['Rolling_Std_3'] = df['Price'].rolling(3).std()

df = df.dropna()

df.head()